In [1]:
# import torch
import numpy as np
import json
import os
# import cv2
import random
import glob
# import matplotlib.pyplot as plt
# import seaborn as sns

import pandas as pd

# import tqdm

BLUE=(255, 0, 0)
GREEN = (0, 255, 0)
RED = (0, 0, 255)
YELLOW = (0, 255, 255)
PINK = (255, 0, 255)
BLACK = (0, 0, 0)
ORANGE = (0, 127, 255)
CUSTOM = (255,170,170)
COLOR_CLASS = {0: BLUE, 1:GREEN, 2:RED, 3:YELLOW, 4:PINK, 5:BLACK, 6:ORANGE, 7:CUSTOM}

def bb_intersection_over_union(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)

    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

def cocoToAbsoluteBox(cocoBox):
    #xywh -> xyxy
    return [cocoBox[0], cocoBox[1], cocoBox[0]+cocoBox[2], cocoBox[1]+cocoBox[3]]


# matplotlib 
def plot(key, list_bbox_):
    fig = plt.figure(figsize=(15,10))

    plt.hist(list_bbox_[key], color = 'blue', edgecolor = 'black',
            bins = int(len(set(list_bbox_[key]))))

    # Add labels
    plt.title('Histogram of {}'.format(key))
    plt.xlabel(key)
    plt.ylabel('count')
#     plt.savefig("d/{}_distribution.png".format(key))

In [2]:
data = json.load(open('/home/hana/sonnh/kaggle-vin/kaggle-VinBigData-Chest-X-ray-Abnormalities-Detection/runs/test/exp434/last42_predictions.json', 'r'))
print(len(data))
fillter_2_class = pd.read_csv('//home/hana/sonnh/kaggle-vin/predict/submission_023.csv')
# fillter_2_class = pd.read_csv('/home/hana/sonnh/kaggle-vin/predict/submission_check_pb.csv')
# fillter_2_class = pd.read_csv('/media/sonnh/kaggle-vin/runs/yolov5_235_submission.csv')

222125


In [3]:
c = 0
for anno in data:
    if anno['score'] >= 0.005:
        c += 1
print(c)

55647


In [4]:
len(fillter_2_class)

c = 0
for string in fillter_2_class['PredictionString']:
    if '14 0.' in string: 
        c +=1 
#         print(string[string.find('14 0.'):])
print(c)

c = 0
for string in fillter_2_class['PredictionString']:
    if '14 1 0 0 1 1' in string: c += 1
print(c)

1266
1713


In [8]:
res = {}
thong_ke = []
x = []
for anno in data:
#     if anno['score'] < 0.05:
#         continue
#         
    image_id = anno['image_id'][:-6]
    x.append(image_id)
    
    xmin = int(anno['bbox'][0])
    ymin = int(anno['bbox'][1])
    xmax = xmin + int(anno['bbox'][2])
    ymax = ymin + int(anno['bbox'][3])
    
    if anno['category_id'] == 12:
        thong_ke.append(anno['score'])
    
    label = [anno['category_id'], anno['score'], xmin, ymin, xmax, ymax]
    if image_id not in res:
        res[image_id] = [label]
    else:
        res[image_id].append(label)

# for anno in data:
#     image_id = anno['image_id'][:-6]
#     if image_id not in res:
#         print(1)
#         res[image_id] = '14 1 0 0 1 1'
print(len(res))

3000


In [38]:
res_1 = {}
for image_id in res:
    labels = []
    best = []
    for label in res[image_id]:
        if label[0] == 0:
            if best:
                if best[1] < label[1]:
                    best = label
            else:
                best = label
        else:
            labels.append(label)
    
    if best: labels.append(best)
    res_1[image_id] = labels
res = res_1



In [39]:
res_1 = {}
for image_id in res:
    labels = []
    best = []
    for label in res[image_id]:
        if label[0] == 3:
            if best:
                if best[1] < label[1]:
                    best = label
            else:
                best = label
        else:
            labels.append(label)
    if best: labels.append(best)
    res_1[image_id] = labels
res = res_1



In [9]:
res_final = {}
for image_id in res:
    res_final[image_id] = ''
    for anno in res[image_id]:
        res_final[image_id] += '{} {} {} {} {} {} '.format(anno[0], anno[1], anno[2],anno[3], anno[4],anno[5])
res = res_final

In [35]:
for i in range(3000):
    if fillter_2_class.iloc[i]['image_id'] not in res:

        res[fillter_2_class.iloc[i]['image_id']] = '14 1 0 0 1 1'
len(res)

3000

# 2 class

In [10]:
for i in range(3000):
    if fillter_2_class.iloc[i]['PredictionString'] == '14 1 0 0 1 1':
        res[fillter_2_class.iloc[i]['image_id']] = '14 1 0 0 1 1'
    
    if '14 0.' in  fillter_2_class.iloc[i]['PredictionString']:
        string = fillter_2_class.iloc[i]['PredictionString']
        res[fillter_2_class.iloc[i]['image_id']] += ' '
        res[fillter_2_class.iloc[i]['image_id']] += string[string.find('14 0.'):]
print(len(res))

3000


In [11]:
thong_ke.sort()
thong_ke[::-1]

[0.51807,
 0.50293,
 0.49902,
 0.35376,
 0.34912,
 0.29028,
 0.21448,
 0.21045,
 0.18494,
 0.18115,
 0.15515,
 0.14221,
 0.12573,
 0.11743,
 0.11676,
 0.10822,
 0.09558,
 0.09424,
 0.08832,
 0.08057,
 0.07458,
 0.07349,
 0.06726,
 0.06567,
 0.06403,
 0.06012,
 0.05493,
 0.05106,
 0.04904,
 0.04684,
 0.04568,
 0.04279,
 0.04199,
 0.04001,
 0.03403,
 0.03244,
 0.03183,
 0.03076,
 0.03006,
 0.0287,
 0.02786,
 0.02724,
 0.02676,
 0.02617,
 0.02599,
 0.02576,
 0.02563,
 0.02562,
 0.02438,
 0.02434,
 0.02357,
 0.02341,
 0.02257,
 0.02225,
 0.02167,
 0.02051,
 0.02007,
 0.01965,
 0.01875,
 0.01865,
 0.01845,
 0.0179,
 0.01706,
 0.0161,
 0.01601,
 0.01496,
 0.01407,
 0.01357,
 0.01355,
 0.01353,
 0.0135,
 0.01301,
 0.01295,
 0.01282,
 0.01273,
 0.01248,
 0.01209,
 0.01194,
 0.01169,
 0.01159,
 0.01138,
 0.01102,
 0.01097,
 0.01054,
 0.01053,
 0.01042,
 0.01025,
 0.01022,
 0.01021,
 0.00998,
 0.00991,
 0.00987,
 0.0098,
 0.00977,
 0.00944,
 0.00939,
 0.00916,
 0.0091,
 0.00906,
 0.00904,
 0.008

# save

In [12]:
s = pd.DataFrame({'image_id': res.keys(), 'PredictionString': res.values()})
s.to_csv('/home/hana/sonnh/kaggle-vin/kaggle-VinBigData-Chest-X-ray-Abnormalities-Detection/runs/test/exp434/last42_predictions.csv', index = False)


In [15]:
s.head()

,image_id,PredictionString
0,785140281117595d1818b8133d43d3bc,3 0.02454 1979 903 2539 1337 3 0.01724 1214 90...
1,fa3640d131e70778d70164f476b828d9,13 0.40137 804 999 1277 1321 0 0.27808 1590 57...
2,80f3da71b6af9134388dee22f0d386d9,14 1 0 0 1 1
3,4dcaf539b6497ad02f2734e1b78c9678,3 0.34082 1105 982 2102 1284 0 0.30054 1448 51...
4,04cc236166c1ff638ff6200f2f2a397d,0 0.31226 1263 482 1539 780 3 0.27295 948 1102...


In [29]:
s.head()

,image_id,PredictionString
0,785140281117595d1818b8133d43d3bc,3 0.11713 1225 908 1823 1199 11 0.04538 735 12...
1,fa3640d131e70778d70164f476b828d9,0 0.50391 1588 568 1928 924 13 0.49878 781 997...
2,80f3da71b6af9134388dee22f0d386d9,3 0.43628 908 1225 1945 1559 0 0.39771 1271 59...
3,4dcaf539b6497ad02f2734e1b78c9678,3 0.64893 1106 992 2094 1286 0 0.61914 1444 50...
4,04cc236166c1ff638ff6200f2f2a397d,0 0.63623 1255 483 1544 791 3 0.39258 958 1108...


In [55]:
len(s[s['PredictionString'] == '14 1 0 0 1 1'])

1713